In [9]:
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import os

def clean_data(df):
    df.columns = df.iloc[2]
    df = df[3:]
    df = df[:-3]
    return df

def generate_days_series(month, year):
    start_date = f'{year}-{month}-10'
    if month == 12:
        end_date = f'{year+1}-01-09'
    else:
        end_date = f'{year}-{month+1}-09'
    dates = pd.date_range(start=start_date, end=end_date)
    return dates

def clean_concat_dataframes(dataframes: dict):
    for key in dataframes.keys():
        dataframes[key] = clean_data(dataframes[key])
    return pd.concat(dataframes, ignore_index=True)

def open_excel_file_path():
    root = Tk()
    root.attributes('-topmost', True)
    root.withdraw()
    filename = askopenfilename(filetypes=[("Excel files", "*.xlsx")], initialdir=os.getcwd(), title="Select Excel File")
    return filename

def getcredit_card_data():
    filename = open_excel_file_path()
    dataframes = pd.read_excel(filename, sheet_name=None)
    df = clean_concat_dataframes(dataframes)
    df = add_month_column(df)
    return df

def add_month_column(df):
    
    df['תאריך עסקה'] = pd.to_datetime(df['תאריך עסקה'], format='%d-%m-%Y')
    df['תאריך חיוב'] = pd.to_datetime(df['תאריך חיוב'], format='%d-%m-%Y')
    df['סכום חיוב'] = df['סכום חיוב'].astype(float)
    
    df['month'] = pd.to_datetime(df['תאריך חיוב']).dt.to_period('M') - 1
    # df['month'] = df['תאריך חיוב']
    return df

def calculate_monthly_expenses(df):
    # change the month column to a period
    df['month'] = pd.to_datetime(df['תאריך חיוב']).apply(lambda dt: dt.replace(day=10))
    unique_dates = df['month'].unique()
    months = []
    years = []

    for date in unique_dates:
        month = date.month
        year = date.year
        dates = generate_days_series(month, year)
        df_period = df[df['month'] == date]
        month_list = df_period['month'].tolist()
        month_list = month_list[0:len(dates)]
        print(f'{month}-{year}')
        cumulative_sum = [round(df_period[df_period['תאריך עסקה'] <= i]['סכום חיוב'].sum(), 2) for i in dates]
        cumulative_df = pd.DataFrame({'date': dates, 'cumulative_sum': cumulative_sum, 'month': month_list})
        # cumulative_sum = { str(i.date()): }
    return cumulative_df

def plot_monthly_expenses(cumulative_sum):
    pass

In [12]:
import sqlite3
df = getcredit_card_data()
df['month'] = str(df['month'])
cumulative_sum = calculate_monthly_expenses(df)

conn = sqlite3.connect('credit_card.db')

df.to_sql('credit_card', conn, if_exists='append', index=False)
df['month'].iloc[0] 

2-2024


Timestamp('2024-02-10 00:00:00')

In [11]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=list(cumulative_sum.keys()), y=list(cumulative_sum.values())))
fig.update_layout(
    xaxis=dict(title='Date'),
    yaxis=dict(title='Cumulative Sum'),
    title='Cumulative Sum Plot',
    xaxis_tickangle=-45
)
fig.show()

# import matplotlib.pyplot as plt

# plt.plot(list(cumulative_sum.keys()), list(cumulative_sum.values()))
# plt.xlabel('Date')
# plt.ylabel('Cumulative Sum')
# plt.title('Cumulative Sum Plot')
# plt.xticks(rotation=45)  # Rotate the x-axis labels by 45 degrees
# plt.show()



TypeError: 'numpy.ndarray' object is not callable

In [ ]:
import json

file_path = 'cumulative_sum.json'
with open(file_path, 'w') as file:
    json.dump(cumulative_sum, file)
with open('cumulative_sum.json') as f:
        data = json.load(f)

In [ ]:
df = pd.read_excel('transaction-details_export_feb.xlsx', sheet_name=None)
df = clean_concat_dataframes(df)